I'm going to predict the actual temperature & what's our human body feels like in that actual temperature using PyTorch. For that, I have several features called Humidity, Pressures, Wind Speed, Wind Bearing.

In [ ]:
import pandas as pd
import numpy as np
import torch
data = pd.read_csv('../input/szeged-weather/weatherHistory.csv')

# **Without Build-In Functions**

In [ ]:
display(data)

In [ ]:
input = np.array([data['Humidity'], data['Pressure (millibars)'],
                 data['Wind Speed (km/h)'], data['Wind Bearing (degrees)']],
                 dtype="float32").T
print(input)

In [ ]:
target = np.array([data['Apparent Temperature (C)'], data['Temperature (C)']],
                  dtype="float32").T
print(target)

In [ ]:
input = torch.from_numpy(input)
target = torch.from_numpy(target)
print(input)
print(target)

In [ ]:
#declare random weight and bias
w = torch.randn(2, 4, requires_grad=True,dtype=torch.float32)
b = torch.randn(2, requires_grad=True, dtype=torch.float32)

In [ ]:
# return (input*weight)+bias
def SModel(x):
    return x @ w.t() + b

In [ ]:
#mean absolute error
def MAE(t1, t2):
    dif = torch.abs(t1 - t2)
    return torch.sum(dif)/dif.numel()

In [ ]:
for i in range(30001):
    pred = SModel(input)
    loss = MAE(pred, target)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 0.00001
        b -= b.grad * 0.00001
        w.grad.zero_()
        b.grad.zero_()
    if (i%1000==0 or i==10000):
        print("Loss:{} at epoch:{}/30000".format(loss, i))

In [ ]:
#Sample test input for random value
testInput = [[0.94, 1100, 15, 250]]
testInput = torch.tensor(testInput, dtype=torch.float32)
testPred = SModel(testInput)
print(testPred)

# **Using Build-In Functions**

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
tstrain = TensorDataset(input, target)
dltrain = DataLoader(tstrain, batch_size=5, shuffle=True)

In [ ]:
model = nn.Linear(4, 2)
model.bias

In [ ]:
preds = model(input)
lossfn = nn.SmoothL1Loss()
loss = lossfn(preds, target)
op = torch.optim.SGD(model.parameters(), 0.00001)

In [ ]:
def fit(epochs, model, lossfn, op, dltrain):
    for epoch in range(epochs):
        for i, j in dltrain:
            pred = model(i)
            loss = lossfn(pred, j)
            loss.backward()
            op.step()
            op.zero_grad()
        if (epoch%10==0 or epoch==epochs):
            print("Loss:{} at epoch:{}/{}".format(loss, epoch+1, epochs))

In [ ]:
fit(30, model, lossfn, op, dltrain)

In [ ]:
model(torch.tensor([[0.91, 1661, 24, 222]]))